In [ ]:
from datasets import load_dataset

dataset = load_dataset("din0s/asqa")

# print(dataset["dev"][:4]['ambiguous_question'])
# print(dataset['dev'][0]['sample_id'])

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained("google/t5-large-ssm-nq")

model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")

In [ ]:
BATCH_SIZE = 64
t5_closed_book_dict = {}

for i in range(0, len(dataset["dev"]), BATCH_SIZE):

    batch = dataset["dev"][i:i+BATCH_SIZE]
    questions = batch['ambiguous_question']
    
    inputs = tokenizer(questions, return_tensors="pt", padding=True)
    gen_outputs = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"], 
        max_length=100, num_beams=5, 
        early_stopping=True)

    answers = tokenizer.batch_decode(gen_outputs, skip_special_tokens=True)

    id_ans_list = list(zip(batch['sample_id'], answers))
    t5_closed_book_dict.update({str(sample_id) : ans for sample_id, ans in id_ans_list})
    # print(t5_closed_book_dict)

In [ ]:
import json
with open('t5_closed_book_predictions.json', 'w') as fp:
    json.dump(t5_closed_book_dict, fp)